# Set Required Args to Activate Modules

In [1]:
from workflow.scripts.resources import Resources

ModuleNotFoundError: No module named 'linkingtool'

# single resource

In [ ]:
required_args = {
        "config_file_path": 'config/config.yml',
        "province_short_code": 'BC',
        "resource_type": 'wind'
    }

resource_module = Resources(**required_args)

In [ ]:
resource_module.get_CF_timeseries()

In [4]:
# resource_module.get_clusters()
# resource_module.get_cluster_timeseries()

# All resource

In [ ]:
# Define the list of resource types
resource_types = ['solar', 'wind']

# Initialize a list to hold the required arguments for each resource type
required_args_list = []

# Loop through each resource type
for resource in resource_types:
    required_args = {
        "config_file_path": 'config/test_config.yml',
        "province_short_code": 'AB',
        "resource_type": resource
    }
    
    required_args_list.append(required_args)

    # Create an instance of Resources with the required arguments
    resource_module = Resources(**required_args)
    
    # Run the processing for the current resource type
    resource_module.run()

In [ ]:
# data1=solar.get_CF_timeseries()
# data2=solar.get_cell_capacity()
# data3=solar.find_grid_nodes()
# data5=solar.score_cells()
# data=datahandler.from_store('cells') 
vis.get_selected_vs_missed_visuals(cells, 
                              required_args['province_short_code'],
                              lcoe_threshold=20, 
                              CF_threshold=0.15, 
                              capacity_threshold=1,
                              text_box_x=.65,
                              text_box_y=.965,
                              title_x=.65,
                              title_y=1,
                              font_size=8,
                              dpi=400,
                              figsize=(8,6),
                              save=True
                              )

In [13]:
lcoe_threshold=20
wcss_tolr=0.05
filtered_cells=cells[cells['lcoe'] <= lcoe_threshold]
filtered_clusters=all_clusters[all_clusters['lcoe'] <= lcoe_threshold]

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# Define the LCOE threshold
lcoe_threshold = 20  # You can set this to the appropriate value as needed

# Create a figure with 1 row and 2 columns
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the selected cells on the left
filtered_cells.plot(
    column='lcoe',
    ax=ax[0],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    legend_kwds={'label': "Cells LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[0].set_title('Cells')
ax[0].set_xlabel('Longitude')
ax[0].set_ylabel('Latitude')

# Plot the filtered clusters on the right
filtered_clusters.plot(
    column='lcoe',
    ax=ax[1],
    edgecolor='white',
    linewidth=0.2,
    legend=True,
    # legend_kwds={'label': "Cluster LCOE ($/kWh)", 'orientation': "vertical"}
)
ax[1].set_title(f'Representative Clusters [@ WCSS tolerance {wcss_tolr}')
ax[1].set_xlabel('Longitude')
ax[1].set_ylabel('Latitude')

# Set a common title for the entire figure
fig.suptitle(f"{select} - Selected Cells vs their Clusters \n[@ lcoe >={lcoe_threshold} $/kWh]", fontsize=16, fontweight='bold')

# Adjust layout
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjusts the top space to make room for the main title
plt.savefig(f'vis/Solar/cells vs cluster_ALL REGIONS_{select}')
# Show the plots
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Get unique regions from the filtered clusters
unique_regions = filtered_clusters['Region'].unique()

# Loop through each unique region and create plots
for region in unique_regions:
    # Filter clusters and cells based on the current region
    region_filtered_clusters = filtered_clusters[filtered_clusters['Region'] == region]
    total_clusters = max(region_filtered_clusters['Cluster_No'])

    # Prepare a list to collect all dissolved cell indices for the region
    dissolved_cell_indices = []

    # Collect dissolved indices for the current region
    for _, row in region_filtered_clusters.iterrows():
        dissolved_cell_indices.extend(list(solar.solar_clusters_nt.dissolved_indices[row.Region][row.Cluster_No]))

    # Extract the dissolved cells based on the collected indices
    dissolved_cells_region = cells.loc[dissolved_cell_indices]
    total_cells = len(dissolved_cells_region)

    # Create a figure with a single row and two columns
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))

    # Plot all filtered clusters for the current region on the left
    cmap = "plasma"  # Changed colormap for better visibility
    region_filtered_clusters.plot(
        column='lcoe',
        edgecolor='gray',
        linewidth=0.8,
        ax=ax[0],
        cmap=cmap,
        legend=False  # Disable the default legend
    )
    
    # Create a colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=region_filtered_clusters['lcoe'].min(), vmax=region_filtered_clusters['lcoe'].max()))
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax[0], orientation='vertical')
    cbar.set_label("Filtered Clusters' LCOE (kWh)")

    ax[0].set_title(f'Filtered Clusters ({total_clusters}) for Region: {region}')
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')

    # Plot all dissolved cells for the current region on the right
    dissolved_cells_region.plot(
        column='lcoe',
        edgecolor='grey',
        ax=ax[1],
        linewidth=0.6,
        alpha=0.7,  # Slightly transparent for better visibility
        cmap=cmap,
        legend=False
    )

    # Create a colorbar for the right plot
    sm_right = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=dissolved_cells_region['lcoe'].min(), vmax=dissolved_cells_region['lcoe'].max()))
    sm_right.set_array([])
    cbar_right = fig.colorbar(sm_right, ax=ax[1], orientation='vertical')
    cbar_right.set_label("Dissolved Cells LCOE (kWh)")

    ax[1].set_title(f'Dissolved Cells ({total_cells}) for Region: {region}')
    ax[1].set_xlabel('Longitude')
    ax[1].set_ylabel('Latitude')

    # Set a common title for the figure
    fig.suptitle(f"Cells vs Representative Clusters (@ WCSS tolerance ={wcss_tolr}) \n {region}, {select} [@ LCOE <= {lcoe_threshold} $/kWh]", fontsize=16)

    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'vis/Solar/cells_vs_cluster_{region}.png')  # Added .png extension for clarity
    plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Get unique regions from the filtered clusters
unique_regions = filtered_clusters['Region'].unique()

# Loop through each unique region and create plots
for region in unique_regions:
    # Filter clusters and cells based on the current region
    region_filtered_clusters = filtered_clusters[filtered_clusters['Region'] == region]
    total_clusters=max(region_filtered_clusters['Cluster_No'])

    # Prepare a list to collect all dissolved cell indices for the region
    dissolved_cell_indices = []

    # Collect dissolved indices for the current region
    for _, row in region_filtered_clusters.iterrows():
        dissolved_cell_indices.extend(list(solar.solar_clusters_nt.dissolved_indices[row.Region][row.Cluster_No]))

    # Extract the dissolved cells based on the collected indices
    dissolved_cells_region = cells.loc[dissolved_cell_indices]
    total_cells=len(dissolved_cells_region)

    # Create a figure with 1 row and 2 columns
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Plot all filtered clusters for the current region on the left
    cmap_left = "viridis"  # Specify your desired colormap here
    region_filtered_clusters.plot(
        column='lcoe',
        edgecolor='gray',
        linewidth=0.8,
        ls=":",
        ax=ax[0],
        cmap=cmap_left,  # Use the specified colormap
        legend=False  # Disable the default legend
    )

    # Create a divider for the left plot to add colorbar
    divider_left = make_axes_locatable(ax[0])
    cax_left = divider_left.append_axes("right", size="5%", pad=0.1)  # Adjust size and padding as needed

    # Create a colorbar
    sm_left = plt.cm.ScalarMappable(cmap=cmap_left, norm=plt.Normalize(vmin=region_filtered_clusters['lcoe'].min(), vmax=region_filtered_clusters['lcoe'].max()))
    sm_left.set_array([])
    cbar_left = fig.colorbar(sm_left, cax=cax_left)
    cbar_left.set_label("Filtered Clusters' LCOE (kWh)")

    ax[0].set_title(f'Filtered Clusters ({total_clusters}) for Region: {region}')
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')

    # Plot all dissolved cells for the current region on the right
    cmap_right = "viridis"  # Specify your desired colormap here
    dissolved_cells_region.plot(
        column='lcoe',
        edgecolor='grey',
        ax=ax[1],
        linewidth=0.6,
        ls=":",
        alpha=1,  # Adjust transparency for better visualization
        cmap=cmap_right,  # Use the specified colormap
        legend=False  # Disable the default legend
    )

    # Create a divider for the right plot to add colorbar
    divider_right = make_axes_locatable(ax[1])
    cax_right = divider_right.append_axes("right", size="5%", pad=0.1)  # Adjust size and padding as needed

    # Create a colorbar
    sm_right = plt.cm.ScalarMappable(cmap=cmap_right, norm=plt.Normalize(vmin=dissolved_cells_region['lcoe'].min(), vmax=dissolved_cells_region['lcoe'].max()))
    sm_right.set_array([])
    cbar_right = fig.colorbar(sm_right, cax=cax_right)
    cbar_right.set_label("Dissolved Cells LCOE (kWh)")

    ax[1].set_title(f'Dissolved Cells ({total_cells}) for Region: {region}')
    ax[1].set_xlabel('Longitude')
    ax[1].set_ylabel('Latitude')

    # Set a common title for the figure
    fig.suptitle(f"Cells vs Representative Clusters ( @ WCSS tolerance ={wcss_tolr}) \n {region},{select} [@ LCOE <= {lcoe_threshold} $/kWh]", fontsize=16)

    # Adjust layout
    plt.tight_layout()
    plt.savefig(f'vis/Solar/cells vs cluster_{region}')
    # Show the plots for the current region
    plt.show()
